<a href="https://colab.research.google.com/github/Ofiktana/well-offset-data/blob/main/Well_Offset_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
def print_header(text, mode):
  print('')
  print('')
  print(100 * ">")
  if mode == 'upper':
    print(text.upper())
  elif mode == 'proper':
    print(text.title())
  print(100 * "<")
  print('')


In [ ]:
def phase_rewrite(text):
  # Make phase descriptions more readable

  text = text.replace('<Code2>','')

  while text[-1] in [' ',',']:
    text = text[:-1]

  return text

def str_cleanup(text):

  # Drop special characters in strings to make them more readable

  checklist = ''',"[']{}'''
  new_text = ''

  for i in text:
    if i not in checklist:
      new_text += i

  return new_text

  # Compile dictionary of problem relevant keywords in DDR

issues = {
    'losses': ['lost circulation','losses','no returns','total losses','partial losses'],
    'stick-slip': ['excessive vibration','erratic','erratic torque'],
    'downhole tool damage': ['broken cone','ringed out'],
    'side-track': ['sidetrack', 'sidetracked', 'side-track','whipstock'],
    'tight spots': ['tight spots','overpull', 'ovp','HUD', 'worked string'],
    'stuck pipe': ['stuck', 'jarred'],
    'well control': ['gas-cut', 'gas cut', 'influx', 'pit gain'],
    'bit-balling': ['balled up', 'balled', 'balling','gumbo'],
    'fishing': ['fish', 'fishing','overshot','grapple'],
    'wellbore instability': ['cavings','hole collapse'],
    'casing': ['casing string','n80','l80','k55','j55'],
    'casing_point':['setting depth','set depth','landed casing','shoe'],
    'total_depth':['section td', 'final td', 'finaltd','sectiontd'],
    'drilling': ['drilled', 'drilling', 'drill ahead', 'drilled ahead'],
    'lithology': ['% shale','% sand','lithology','lignite', 'shale -', 'sand -'],
    'lot': ['performed lot', 'performed fit','leak-off', 'formation integrity','leak off', 'carried out lot','carried out fit']
}

def issues_check(comment):

  if len(comment) > 0:
    msg = []

    for key,value in issues.items():
      for word in value:
        # Loop through dictionary words to find words appearing in the timelog, collate dictionary keys for applicable values and save to 'msg' list.

        if word in comment:
          msg.append(key)

    if len(msg) > 0:
      return str_cleanup(str(set(msg)))
    else:
      return ''
  else:
    return ''


In [ ]:
def drilling_parameters(df, unit):

  for i, row in df.iterrows():
    parameters = list(re.findall(rf'\d+\s*{unit}', row['comments']))

    if 'drilling' in row['remarks'] and len(parameters) > 0:

      parameter = str_cleanup(parameters[0])

      df.loc[i, unit] = float(parameter.replace(unit,''))
    else:
      df.loc[i, unit] = 0.0

def get_drilling_parameters(df):
  drilling_parameters(df, 'rpm')
  drilling_parameters(df, 'gpm')
  drilling_parameters(df, 'psi')
  drilling_parameters(df, 'kips')
  drilling_parameters(df, 'klbs')

  for i, row in df.iterrows():
    kips = row['kips']
    klbs = row['klbs']

    if kips == 0.0 and klbs > 0:
      df.loc[i, 'wob'] = klbs
    else:
      df.loc[i, 'wob'] = kips

  df.drop('kips', axis=1, inplace = True)
  df.drop('klbs', axis=1, inplace = True)


In [ ]:
def extract_well_data(filepath):

  # Import well timelog csv file

  well_data = pd.read_csv(filepath)

  # Transform character case and data type

  well_data['comments'] = well_data['Com'].str.lower()

  well_data['comments'] = well_data['comments'].astype(str)

  well_data['Phase'] = well_data['Phase'].astype(str)

  return well_data


In [ ]:
def transform_well_data(well_data):
  well_data['remarks'] = well_data['comments'].apply(lambda x: issues_check(x))
  well_data['Phase'] = well_data['Phase'].apply(phase_rewrite)
  get_drilling_parameters(well_data)

def load_well_data(well_name, well_data):
  print_header(f'{well_name} timelog (all)','upper')
  display(well_data[['Rpt #','Phase','Unschd Type','Start Date','Dur (hr)','End Date','Start Depth (ftKB)','End Depth (ftKB)','Com','wob','rpm','gpm','psi','remarks']])

  # #print Remarks
  # print_header(f'{well_name} timelog (filtered by remarks)','upper')

  # for issue in issues.keys():
  #   print_header(f'{well_name} timelog - remark: {issue}','proper')
  #   display(well_data[well_data['remarks'].str.contains(f"{issue}")][['Unschd Type','Start Date','Dur (hr)','End Date','Start Depth (ftKB)','End Depth (ftKB)','Com','remarks']])




In [ ]:
def well_data_pipeline(well_name, wellfile):
  well_data = extract_well_data(wellfile)
  transform_well_data(well_data)
  load_well_data(well_name, well_data)

well_name =  'Oben 48'
wellfile_csv = 'Oben 48 Well Ops Timelog.csv'

well_data_pipeline(well_name, wellfile_csv)



>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
OBEN 48 TIMELOG (ALL)
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<



,Rpt #,Phase,Unschd Type,Start Date,Dur (hr),End Date,Start Depth (ftKB),End Depth (ftKB),Com,wob,rpm,gpm,psi,remarks
0,1.0,Rig Move,Planned,8/19/2019 0:00,6.0,8/19/2019 6:00,0.00,0.00,Cont'd to wait on daylight,0.0,0.0,0.0,0.0,
1,1.0,Rig Move,Planned,8/19/2019 6:00,0.5,8/19/2019 6:30,0.00,0.00,Held PJSM,0.0,0.0,0.0,0.0,
2,1.0,Rig Move,Planned,8/19/2019 6:30,11.5,8/19/2019 18:00,0.00,0.00,Welder cont'd to dislodge intermediate tank an...,0.0,0.0,0.0,0.0,
3,1.0,Rig Move,Planned,8/19/2019 18:00,6.0,8/20/2019 0:00,0.00,0.00,Wait on Daylight,0.0,0.0,0.0,0.0,
4,2.0,Rig Move,Planned,8/20/2019 0:00,6.0,8/20/2019 6:00,0.00,0.00,Cont’d to wait on daylight,0.0,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,196.0,Upper Completions,Planned,2/29/2020 19:30,0.5,2/29/2020 20:00,"11,177.00","11,177.00",Flushed & cleaned THS flange + Hanger neck. In...,0.0,0.0,0.0,0.0,
1539,196.0,Upper Completions,Planned,2/29/2020 20:00,3.0,2/29/2020 23:00,"11,177.00","11,177.00","PU, aligned & NU Xmas tree (feed PDHG & TRSCSS...",0.0,0.0,0.0,0.0,
1540,196.0,Upper Completions,Planned,2/29/2020 23:00,1.0,3/1/2020 0:00,"11,177.00","11,177.00",Test Flange + Hanger neck seals to 5000psi x 1...,0.0,0.0,0.0,0.0,
1541,197.0,Upper Completions,Planned,3/1/2020 0:00,3.0,3/1/2020 3:00,"11,177.00","11,177.00","MU 5-1/8"" flange x 2"" WECO Union on Xmas tree....",0.0,0.0,0.0,0.0,
